# Setup project

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df_2010 = pd.read_csv('reces_2010_canton_ville_D.csv', sep=';', low_memory=False)

In [3]:
df_cantons = pd.read_csv('cantons_cleaned.csv')

In [51]:
# used to filter the cantons ds csv (find it on the drive)
DEPARTEMENTS = ['16',
                '17',
                '19',
                '23',
                '24',
                '33',
                '40',
                '47',
                '64',
                '79',
                '86',
                '87']
DEPARTEMENTS_INT = [int(numeric_string) for numeric_string in DEPARTEMENTS]

In [52]:
# columns to keep in each pass
COLUMNS_TO_KEEP = [
    "annee",
    "departement",
    "AGED",
    "ASCEN",
    "CATL",
    "CATPC",
    "CHFL",
    "CMBL",
    "DIPL",
    "EMPL",
    "ETUD",
    "GARL",
    "HLML",
    "ILT",
    "IMMI",
    "INAI",
    "INFAM",
    "LIENF",
    "MATR",
    "MOCO",
    "MODV",
    "NA17",
    "NAIDT",
    "NBPI",
    "NENFR",
    "NPERR",
    "ORIDT",
    "RECH",
    "SANI",
    "SFM",
    "STOCD",
    "SURF",
    "TACTD16",
    "TP",
    "TRANS",
    "TYPL",
    "VOIT",
]

In [63]:
# columns that need to be transformed to dummies
TO_DUMMIES = [
    "ASCEN",
    "CATL",
    "CHFL",
    "CMBL",
    "DIPL",
    "EMPL",
    "GARL",
    "HLML",
    "ILT",
    "INFAM",
    "LIENF",
    "NA17",
    "NBPI",
    "NENFR",
    "NPERR",
    "RECH",
    "SANI",
    "SFM",
    "STOCD",
    "SURF",
    "TP",
    "TRANS",
    "TYPL",
    "VOIT",
]

# Study dataset

In [11]:
df_2010.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4311337 entries, 0 to 4311336
Data columns (total 89 columns):
 #   Column       Dtype  
---  ------       -----  
 0   CANTVILLE    int64  
 1   NUMMI        object 
 2   AEMMR        object 
 3   AGED         int64  
 4   AGER20       int64  
 5   AGEREV       int64  
 6   AGEREVQ      int64  
 7   ANAI         int64  
 8   ANEMR        object 
 9   APAF         object 
 10  ARM          object 
 11  ASCEN        object 
 12  BAIN         object 
 13  BATI         object 
 14  CATIRIS      object 
 15  CATL         object 
 16  CATPC        int64  
 17  CHAU         object 
 18  CHFL         object 
 19  CHOS         object 
 20  CLIM         object 
 21  CMBL         object 
 22  COUPLE       int64  
 23  CS1          int64  
 24  CUIS         object 
 25  DEPT         int64  
 26  DEROU        object 
 27  DIPL         object 
 28  DNAI         object 
 29  EAU          object 
 30  EGOUL        object 
 31  ELEC         object 
 32

In [6]:
df_2010.head()

,CANTVILLE,NUMMI,AEMMR,AGED,AGER20,AGEREV,AGEREVQ,ANAI,ANEMR,APAF,...,TP,TRANS,TRIRIS,TYPC,TYPFC,TYPL,TYPMC,TYPMR,VOIT,WC
0,101,1,6,58,64,57,55,1950,05,0,...,Z,Z,ZZZZZZ,2,Z,1,1,12,1,Z
1,101,2,6,55,54,54,50,1955,05,2,...,1,4,ZZZZZZ,1,2,1,4,41,3,Z
2,101,2,6,55,54,54,50,1955,05,1,...,1,4,ZZZZZZ,1,2,1,4,41,3,Z
3,101,3,9,45,54,44,40,1963,01,0,...,1,2,ZZZZZZ,2,Z,1,1,12,1,Z
4,101,4,9,58,64,57,55,1953,01,0,...,1,5,ZZZZZZ,3,Z,2,1,11,1,Z


In [7]:
df_2010['CANTVILLE'].value_counts()

CANTVILLE
6999    183960
3199    168310
3399     91213
4299     64985
3898     61222
         ...  
8103       158
913        137
3893       116
4398        42
8140        35
Name: count, Length: 1082, dtype: int64

In [8]:
df_cantons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Département      255 non-null    int64
 1   Canton-ou-ville  255 non-null    int64
dtypes: int64(2)
memory usage: 4.1 KB


# Work on dataset

## Test

In [36]:
# replace next df by this to try pipeline
test = df_2010.sample(n=100)

## Work

In [37]:
# change df used by current_df to test
current_df = df_2010

In [38]:
current_df['annee'] = 2010

In [41]:
# filter year and extract rows that have a cantville in the Nouvelle-Aquitaine cantons dataset
filtered_df = current_df[current_df['CANTVILLE'].isin(df_cantons['Canton-ou-ville'])]

In [45]:
# merge filtered and cantons df to get cantons departement for each rows of the filtered df
merged_df = pd.merge(filtered_df, df_cantons, how='left', left_on='CANTVILLE', right_on='Canton-ou-ville')

In [48]:
# rename departement column to match syntax rules
merged_df = merged_df.rename(columns={
    'Département': 'departement'
})

In [53]:
# filter again the df to keep allowed columns
merged_df = merged_df.loc[:, COLUMNS_TO_KEEP]

In [58]:
merged_df['departement'].value_counts()

departement
33    241408
64    105942
87     78271
47     54666
40     50070
24     31822
19     27463
23     15640
Name: count, dtype: int64

In [60]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 605282 entries, 0 to 605281
Data columns (total 37 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   annee        605282 non-null  int64 
 1   departement  605282 non-null  int64 
 2   AGED         605282 non-null  int64 
 3   ASCEN        605282 non-null  object
 4   CATL         605282 non-null  object
 5   CATPC        605282 non-null  int64 
 6   CHFL         605282 non-null  object
 7   CMBL         605282 non-null  object
 8   DIPL         605282 non-null  object
 9   EMPL         605282 non-null  object
 10  ETUD         605282 non-null  int64 
 11  GARL         605282 non-null  object
 12  HLML         605282 non-null  object
 13  ILT          605282 non-null  object
 14  IMMI         605282 non-null  int64 
 15  INAI         605282 non-null  int64 
 16  INFAM        605282 non-null  object
 17  LIENF        605282 non-null  object
 18  MATR         605282 non-null  int64 
 19  MO

In [ ]:
TO_DUMMIES = [
    "ASCEN",
    "CATL",
    "CHFL",
    "CMBL",
    "DIPL",
    "EMPL",
    "GARL",
    "HLML",
    "ILT",
    "INFAM",
    "LIENF",
    "NA17",
    "NBPI",
    "NENFR",
    "NPERR",
    "RECH",
    "SANI",
    "SFM",
    "STOCD",
    "SURF",
    "TP",
    "TRANS",
    "TYPL",
    "VOIT",
]